In [372]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [374]:
import numpy as np
filename = '../content/drive/MyDrive/Colab Notebooks/Monitors/dades.txt'
data = np.loadtxt(filename, delimiter=',', dtype=str)
print(data)

[['deto' 'ona' 'gasull roca paula' '-' 'M T X B A' 'A']
 ['albert' '-' 'joaquim biel' '-' 'X M T A B' '-']
 ['biel' '-' 'helena' '-' 'B T X M A' '-']
 ['ona' 'deto merce iu tibau helena' '-' '-' 'M X B T A' '-']
 ['gasull' '-' '-' '-' 'B X M A T' '-']
 ['paula' 'ona tibau' 'gasull' '-' 'X B M T A' '-']
 ['suri' 'massana roca ense' 'helena joaquim' '-' 'X T M B A' 'A']
 ['merce' 'iu paula ona' ' joaquim albert' '-' 'X M B T A' '-']
 ['tibau' 'ona suri merce' 'joaquim' '-' 'M X A T B' 'A']
 ['ense' 'suri susi roca' 'joaquim' '-' 'M X A T B' 'A T']
 ['helena' 'ona merce' 'joaquim gasull biel' '-' 'X B T M A' '-']
 ['iu' 'paula merce ona' 'joaquim' '-' 'X M T B A' '-']
 ['roca' 'ense suri' '-' '-' 'T X B M A' 'M A']]


In [ ]:
def startup_person_list(data):
  persons = []
  for entry in data:
    temp_p = person(entry)
    persons.append(temp_p)
  return persons

def startup_group_list(names, sizes):
  groups = []
  for i in range(len(names)):
    temp_g = group(names[i], [], sizes[i])
    groups.append(temp_g)
  return groups

def get_random_index(rang, exclude):
  test_list = range(5)
  res = random.choice([ele for ele in test_list if ele != exclude])
  return res

In [362]:
import random

number_groups = 5
number_persons = 13

g_names = ['A', 'X', 'M', 'T', 'B']
#g_names = ['1','2','3','4','5','6','7']
g_sizes = [2, 3, 3, 3, 2]
#g_sizes = [2,2,2,2,2,2,2]

absolute_score = 3*(max(g_sizes)-1) + int(number_groups/2)+1

class person:
  def __init__(self, data_array):
    self.p_name = data_array[0]
    self.pos_person = data_array[1].split(" ")
    self.neg_person = data_array[2].split(" ")
    self.imp_person = data_array[3].split(" ")
    self.group_pref = data_array[4].split(" ")
    self.imp_group = data_array[5].split(" ")
    self.max_score = 3*len(self.pos_person) + int(number_groups/2)+1
    if self.max_score > absolute_score: 
      self.max_score = absolute_score
    self.score = 0

  #Getters: 
  def p_info(self):
    print(self.p_name, self.pos_person, self.neg_person, self.imp_person, self.group_pref, self.max_score)
  def g_info(self):
    return(self.p_name,self.score, self.max_score)

  #Funcions:
  def p_score(self, curr_group, curr_integrants, n_groups):
    rel_score = 0

    #Positive and negative persons:
    rel_score += 4*sum(el.p_name in self.pos_person for el in curr_integrants)
    rel_score -= 7*sum(el.p_name in self.neg_person for el in curr_integrants)

    #Group:
    rel_score += 2*(int(n_groups/2) - self.group_pref.index(curr_group))

    if (curr_group in self.imp_group):
      rel_score = -10

    #self.score = (rel_score/(self.max_score+n_groups/2))*100
    self.score = rel_score
    return self.score 
  
class group:
  def __init__(self, name, persons, size):
    self.size = size
    self.g_name = name
    self.g_persons = persons
    self.general_score = 0

  #Setters:
  def add_person(self, person):
    self.g_persons.append(person)

  #Getters:
  def p_info(self):
    print('Group name: ', self.g_name)
    print('Score: ', self.general_score)
    for person in self.g_persons:
      n, s, ms = person.g_info()
      print('Name: ', n, ' Score: ', s,'/',ms)
  def g_info(self):
    return(self.g_name, [person.g_info() for person in self.g_persons])

  #Funcions
  def gen_scores(self, n_groups):
    self.general_score = 0
    for person in self.g_persons:
      self.general_score += person.p_score(self.g_name, self.g_persons, n_groups)
    self.general_score = self.general_score/self.size
    return self.general_score
  
  def get_scores(self):
    p_scores_list = []
    for person in self.g_persons:
       p_scores_list.append(person.score)
    return p_scores_list

  def swap_person(self, index):
    ret = self.g_persons.pop(index)
    return ret 

class entity:
  def __init__(self, groups, persons):
    self.e_persons = persons
    self.e_groups = groups
    self.e_group_scores = []
  
  def p_info(self):
    print([group.p_info() for group in self.e_groups])

  def gen_random_distribution(self):
    aux = self.e_persons.copy()
    random.shuffle(aux)
    for i in range(len(self.e_groups)):
      for j in range(self.e_groups[i].size):
        self.e_groups[i].add_person(aux.pop())

  def update_data(self):
    self.e_group_scores.clear()
    for i in range(len(self.e_groups)):
      self.e_group_scores.append(self.e_groups[i].gen_scores(len(groups)))

  def swap_group_member(self, index):
    saddest_g = (self.e_groups[index].get_scores())
    min_v = min(saddest_g)
    ind = saddest_g.index(min_v)
    aux = self.e_groups[index].swap_person(ind)
    return aux 

  def find_optimal_sol(self, iterations, thresh_hold):

    for i in range(iterations):
      #Find saddest group and swaping group:
      min_v = min(self.e_group_scores)
      min_i = self.e_group_scores.index(min_v)
      swap_i = get_random_index(len(self.e_groups),min_i)
      #Swap 1:
      aux_1 = self.swap_group_member(min_i)
      #Swap 2:
      aux_2 = self.swap_group_member(swap_i)
      self.e_groups[min_i].add_person(aux_2)
      self.e_groups[swap_i].add_person(aux_1)
      e.update_data()
      #Save good result
      if sum(self.e_group_scores) >= thresh_hold:
        print('')
        print('Iteracio num:',i,'==============')
        print('Group score:',sum(self.e_group_scores))
        self.p_info()

In [382]:
groups = startup_group_list(g_names, g_sizes)
persons = startup_person_list(data)
e = entity(groups, persons)
e.gen_random_distribution()
e.update_data()
temp = e.find_optimal_sol(1000,21)


Iteracio num: 469 ==============
Group score: 22.333333333333332
Group name:  A
Score:  -3.0
Name:  albert  Score:  -2 / 6
Name:  helena  Score:  -4 / 9
Group name:  X
Score:  9.333333333333334
Name:  iu  Score:  12 / 9
Name:  merce  Score:  12 / 9
Name:  paula  Score:  4 / 9
Group name:  M
Score:  6.666666666666667
Name:  ona  Score:  8 / 9
Name:  deto  Score:  8 / 6
Name:  ense  Score:  4 / 9
Group name:  T
Score:  5.333333333333333
Name:  roca  Score:  8 / 9
Name:  suri  Score:  6 / 9
Name:  tibau  Score:  2 / 9
Group name:  B
Score:  4.0
Name:  biel  Score:  4 / 6
Name:  gasull  Score:  4 / 6
[None, None, None, None, None]

Iteracio num: 915 ==============
Group score: 21.0
Group name:  A
Score:  -3.0
Name:  iu  Score:  -4 / 9
Name:  albert  Score:  -2 / 6
Group name:  X
Score:  6.666666666666667
Name:  helena  Score:  8 / 9
Name:  merce  Score:  8 / 9
Name:  paula  Score:  4 / 9
Group name:  M
Score:  9.333333333333334
Name:  ona  Score:  12 / 9
Name:  tibau  Score:  8 / 9
Name: 